# 🎯 Création et Gestion de Stopwords Personnalisés

**Module 2 - Preprocessing et Tokenisation**

Ce notebook vous apprend à créer, gérer et optimiser vos propres listes de stopwords selon vos besoins spécifiques. Parce qu'une liste universelle ne convient jamais à tous les cas !

## 📋 Plan du Notebook

1. **Pourquoi des Stopwords Personnalisés ?**
2. **Méthodes d'Identification Automatique**
3. **Stopwords par Domaine d'Application**
4. **Gestionnaire Avancé de Stopwords**
5. **Validation et Tests**
6. **Optimisation par Fréquence**
7. **Cas d'Usage Pratiques**
8. **Export et Réutilisation**

## 📦 Installation et Imports

In [ ]:
# Installation des packages nécessaires
# !pip install nltk spacy matplotlib seaborn pandas wordcloud scikit-learn
# !python -m spacy download fr_core_news_sm

In [ ]:
import nltk
import spacy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter, defaultdict
from wordcloud import WordCloud
import re
import json
from typing import List, Set, Dict, Tuple
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Configuration matplotlib
plt.style.use('seaborn-v0_8')
sns.set_palette("Set2")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11

print("✅ Tous les imports réalisés avec succès !")

In [ ]:
# Chargement des ressources de base
nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Chargement spaCy
try:
    nlp = spacy.load("fr_core_news_sm")
    print("✅ Modèle spaCy français chargé")
except OSError:
    print("⚠️ Modèle spaCy français non trouvé, utilisation des fonctions de base")
    nlp = None

# Stopwords de base
stopwords_base_fr = set(stopwords.words('french'))
print(f"📚 {len(stopwords_base_fr)} stopwords français de base chargés")

## 1. 🤔 Pourquoi des Stopwords Personnalisés ?

### Les limites des listes standard :
- **Trop générales** : ne tiennent pas compte de votre domaine
- **Parfois incomplètes** : manquent des mots fréquents dans votre contexte
- **Parfois trop strictes** : suppriment des mots importants pour votre analyse

### Exemples concrets :

In [ ]:
# Exemples où les stopwords standard posent problème
exemples_problematiques = {
    "E-commerce - Avis produit": {
        "texte": "Ce produit est vraiment bien. Service client excellent. Livraison rapide. Je recommande ce site.",
        "probleme": "'produit', 'service', 'client' sont fréquents mais informatifs",
        "mots_a_ajouter": ["vraiment", "bien", "excellent", "rapide"]
    },
    
    "Médical - Comptes-rendus": {
        "texte": "Le patient présente des symptômes. Examen clinique normal. Traitement prescrit.",
        "probleme": "'patient', 'examen', 'traitement' sont techniques mais répétitifs",
        "mots_a_ajouter": ["patient", "examen", "clinique", "traitement"]
    },
    
    "Réseaux sociaux - Tweets": {
        "texte": "Salut les amis ! MDR ce truc est ouf ! Genre trop cool lol",
        "probleme": "Argot et abréviations non couverts par les listes standard",
        "mots_a_ajouter": ["salut", "amis", "mdr", "lol", "genre", "truc", "ouf"]
    },
    
    "Technique - Documentation": {
        "texte": "Cette fonction permet de configurer le système. Voir documentation pour plus d'informations.",
        "probleme": "'fonction', 'système', 'documentation' sont répétitifs en tech",
        "mots_a_ajouter": ["fonction", "permet", "configurer", "système", "voir", "documentation"]
    }
}

print("🎯 EXEMPLES DE CAS PROBLÉMATIQUES\n")

for domaine, info in exemples_problematiques.items():
    print(f"📂 {domaine}")
    print(f"   Texte : {info['texte']}")
    print(f"   ⚠️  Problème : {info['probleme']}")
    print(f"   💡 Stopwords à ajouter : {', '.join(info['mots_a_ajouter'])}")
    print()

## 2. 🔍 Méthodes d'Identification Automatique

Comment identifier automatiquement les candidats stopwords dans vos données ?

In [ ]:
class DetecteurStopwords:
    """Classe pour détecter automatiquement les stopwords candidats."""
    
    def __init__(self):
        self.frequences = Counter()
        self.documents_count = 0
        self.document_frequencies = defaultdict(int)
    
    def analyser_corpus(self, textes: List[str]) -> None:
        """
        Analyse un corpus pour calculer les fréquences.
        
        Args:
            textes: Liste de textes à analyser
        """
        self.documents_count = len(textes)
        
        for texte in textes:
            # Tokenisation simple
            mots = re.findall(r'\w+', texte.lower())
            
            # Comptage global
            self.frequences.update(mots)
            
            # Comptage par document (pour IDF)
            mots_uniques = set(mots)
            for mot in mots_uniques:
                self.document_frequencies[mot] += 1
    
    def candidats_par_frequence(self, seuil_frequence: int = 10, 
                               pourcentage_top: float = 0.05) -> List[Tuple[str, int]]:
        """
        Identifie les candidats stopwords par fréquence élevée.
        
        Args:
            seuil_frequence: Fréquence minimale
            pourcentage_top: Pourcentage des mots les plus fréquents
        
        Returns:
            Liste des candidats avec leur fréquence
        """
        # Mots très fréquents
        mots_frequents = [(mot, freq) for mot, freq in self.frequences.most_common() 
                         if freq >= seuil_frequence]
        
        # Top X% des mots
        nb_top = max(1, int(len(self.frequences) * pourcentage_top))
        top_mots = self.frequences.most_common(nb_top)
        
        # Combinaison des critères
        candidats = list(set(mots_frequents + top_mots))
        candidats.sort(key=lambda x: x[1], reverse=True)
        
        return candidats
    
    def candidats_par_distribution(self, seuil_presence: float = 0.7) -> List[Tuple[str, float]]:
        """
        Identifie les candidats par distribution dans les documents.
        
        Args:
            seuil_presence: Pourcentage minimum de documents contenant le mot
        
        Returns:
            Liste des candidats avec leur taux de présence
        """
        candidats = []
        
        for mot, doc_freq in self.document_frequencies.items():
            taux_presence = doc_freq / self.documents_count
            if taux_presence >= seuil_presence:
                candidats.append((mot, taux_presence))
        
        candidats.sort(key=lambda x: x[1], reverse=True)
        return candidats
    
    def candidats_par_longueur(self, longueur_max: int = 3) -> List[Tuple[str, int]]:
        """
        Identifie les mots courts très fréquents (souvent des stopwords).
        
        Args:
            longueur_max: Longueur maximale des mots
        
        Returns:
            Liste des candidats courts avec leur fréquence
        """
        candidats = [(mot, freq) for mot, freq in self.frequences.most_common() 
                    if len(mot) <= longueur_max and freq >= 5]
        
        return candidats[:20]  # Top 20
    
    def rapport_analyse(self) -> Dict:
        """
        Génère un rapport d'analyse complet.
        
        Returns:
            Dictionnaire avec les statistiques du corpus
        """
        total_mots = sum(self.frequences.values())
        vocabulaire_unique = len(self.frequences)
        
        return {
            'documents_count': self.documents_count,
            'total_mots': total_mots,
            'vocabulaire_unique': vocabulaire_unique,
            'mots_par_document': total_mots / self.documents_count if self.documents_count > 0 else 0,
            'diversite_lexicale': vocabulaire_unique / total_mots if total_mots > 0 else 0,
            'top_10_mots': self.frequences.most_common(10)
        }

print("✅ Classe DetecteurStopwords créée")

In [ ]:
# Test du détecteur sur un corpus d'exemple
corpus_ecommerce = [
    "Ce produit est vraiment excellent, je le recommande à tous mes amis et ma famille.",
    "Service client très professionnel, livraison rapide, emballage soigné, parfait.",
    "Produit conforme à la description, qualité au rendez-vous, très satisfait de mon achat.",
    "Prix un peu élevé mais la qualité justifie largement cet investissement, je recommande.",
    "Commande reçue rapidement, produit de bonne qualité, service client réactif.",
    "Très bon rapport qualité prix, produit fiable, livraison dans les temps.",
    "Service après vente excellent, problème résolu rapidement, équipe très professionnelle.",
    "Produit exactement comme décrit, emballage parfait, livraison ultra rapide.",
    "Qualité exceptionnelle, service client à l'écoute, je recommande vivement ce site.",
    "Achat sans problème, produit conforme, délai de livraison respecté, parfait."
]

# Analyse du corpus
detecteur = DetecteurStopwords()
detecteur.analyser_corpus(corpus_ecommerce)

# Rapport d'analyse
rapport = detecteur.rapport_analyse()
print("📊 RAPPORT D'ANALYSE DU CORPUS E-COMMERCE\n")
print(f"Documents analysés : {rapport['documents_count']}")
print(f"Total de mots : {rapport['total_mots']}")
print(f"Vocabulaire unique : {rapport['vocabulaire_unique']}")
print(f"Mots par document : {rapport['mots_par_document']:.1f}")
print(f"Diversité lexicale : {rapport['diversite_lexicale']:.3f}")

print(f"\n🔝 Top 10 des mots les plus fréquents :")
for mot, freq in rapport['top_10_mots']:
    print(f"  '{mot}' : {freq} fois")

In [ ]:
# Identification des candidats stopwords
print("🔍 IDENTIFICATION DES CANDIDATS STOPWORDS\n")

# Par fréquence
candidats_freq = detecteur.candidats_par_frequence(seuil_frequence=3, pourcentage_top=0.1)
print(f"📈 Candidats par fréquence élevée ({len(candidats_freq)}) :")
for mot, freq in candidats_freq[:15]:
    print(f"  '{mot}' : {freq} fois")

# Par distribution
candidats_distrib = detecteur.candidats_par_distribution(seuil_presence=0.6)
print(f"\n📊 Candidats par distribution ({len(candidats_distrib)}) :")
for mot, taux in candidats_distrib[:15]:
    print(f"  '{mot}' : {taux:.1%} des documents")

# Par longueur
candidats_courts = detecteur.candidats_par_longueur(longueur_max=3)
print(f"\n🔤 Mots courts fréquents ({len(candidats_courts)}) :")
for mot, freq in candidats_courts[:15]:
    print(f"  '{mot}' ({len(mot)} lettres) : {freq} fois")

## 3. 🏢 Stopwords par Domaine d'Application

Création de listes spécialisées pour différents domaines.

In [ ]:
# Dictionnaire complet de stopwords par domaine
STOPWORDS_DOMAINES = {
    'e_commerce': {
        'description': 'Commerce électronique, avis clients, produits',
        'mots': {
            # Mots liés au commerce
            'produit', 'article', 'commande', 'achat', 'vente', 'prix', 'euros', 'euro',
            'livraison', 'service', 'client', 'clients', 'boutique', 'magasin', 'site',
            'qualité', 'rapport', 'satisfait', 'satisfaction', 'recommande', 'recommander',
            
            # Adverbes d'intensité fréquents
            'vraiment', 'très', 'super', 'excellent', 'parfait', 'génial', 'top',
            'bien', 'bon', 'bonne', 'mieux', 'meilleur', 'meilleure',
            
            # Mots d'évaluation
            'avis', 'commentaire', 'note', 'étoiles', 'étoile', 'points',
            'expérience', 'test', 'essai', 'utilisation'
        }
    },
    
    'medical': {
        'description': 'Domaine médical, comptes-rendus, diagnostics',
        'mots': {
            # Personnel médical
            'patient', 'patients', 'médecin', 'docteur', 'infirmier', 'infirmière',
            'praticien', 'spécialiste', 'chirurgien',
            
            # Lieux médicaux
            'hôpital', 'clinique', 'cabinet', 'consultation', 'urgence', 'urgences',
            'service', 'département', 'unité',
            
            # Procédures
            'examen', 'diagnostic', 'traitement', 'thérapie', 'intervention',
            'opération', 'chirurgie', 'analyse', 'bilan', 'contrôle',
            
            # Médicaments
            'médicament', 'ordonnance', 'prescription', 'posologie', 'dose',
            'traitement', 'thérapeutique'
        }
    },
    
    'reseaux_sociaux': {
        'description': 'Réseaux sociaux, langage informel, argot',
        'mots': {
            # Salutations informelles
            'salut', 'coucou', 'hello', 'hey', 'yo', 'bjr', 'bsr', 'slt',
            
            # Expressions d'humour/émotion
            'mdr', 'lol', 'ptdr', 'xdr', 'haha', 'hihi', 'lmao', 'rofl',
            
            # Mots de liaison informels
            'genre', 'style', 'comme', 'enfin', 'bref', 'donc', 'alors',
            'bon', 'ben', 'bah', 'euh', 'heu',
            
            # Mots vagues
            'truc', 'machin', 'chose', 'bidule', 'truc', 'trucs',
            
            # Intensificateurs informels
            'trop', 'hyper', 'méga', 'ultra', 'super', 'grave', 'carrément',
            
            # Abréviations courantes
            'jsp', 'jpp', 'jtm', 'jte', 'cv', 'ça va', 'qqun', 'qqch'
        }
    },
    
    'technique': {
        'description': 'Documentation technique, informatique',
        'mots': {
            # Termes techniques génériques
            'système', 'fonction', 'méthode', 'classe', 'objet', 'variable',
            'paramètre', 'configuration', 'option', 'valeur', 'données',
            
            # Actions techniques
            'permet', 'utilise', 'configure', 'définit', 'spécifie', 'indique',
            'exécute', 'lance', 'démarre', 'arrête', 'charge', 'sauvegarde',
            
            # Documentation
            'voir', 'consulter', 'référence', 'documentation', 'manuel', 'guide',
            'exemple', 'note', 'remarque', 'attention', 'important',
            
            # Versions et formats
            'version', 'format', 'type', 'mode', 'niveau', 'statut', 'état'
        }
    },
    
    'juridique': {
        'description': 'Domaine juridique, contrats, lois',
        'mots': {
            # Acteurs juridiques
            'tribunal', 'juge', 'avocat', 'procureur', 'partie', 'parties',
            'demandeur', 'défendeur', 'témoin', 'expert',
            
            # Procédures
            'procédure', 'instance', 'jugement', 'arrêt', 'décision', 'ordonnance',
            'sentence', 'verdict', 'délibéré',
            
            # Documents
            'contrat', 'accord', 'convention', 'clause', 'article', 'alinéa',
            'paragraphe', 'annexe', 'pièce',
            
            # Expressions juridiques
            'considérant', 'attendu', 'vu', 'statuant', 'ordonne', 'déclare',
            'condamne', 'débouté'
        }
    },
    
    'actualites': {
        'description': 'Presse, actualités, journalisme',
        'mots': {
            # Connecteurs journalistiques
            'selon', 'd\'après', 'conformément', 'suite', 'concernant', 'regarding',
            'également', 'notamment', 'particulièrement', 'spécialement',
            
            # Nuances et transitions
            'toutefois', 'cependant', 'néanmoins', 'malgré', 'en dépit',
            'par ailleurs', 'en outre', 'de plus', 'en effet', 'ainsi',
            
            # Temporalité
            'hier', 'aujourd\'hui', 'demain', 'récemment', 'actuellement',
            'désormais', 'dorénavant', 'prochainement',
            
            # Sources et références
            'source', 'déclaration', 'communiqué', 'rapport', 'étude', 'enquête',
            'témoignage', 'interview', 'entretien'
        }
    }
}

print("📚 BIBLIOTHÈQUE DE STOPWORDS PAR DOMAINE\n")
for domaine, info in STOPWORDS_DOMAINES.items():
    print(f"🏢 {domaine.upper().replace('_', ' ')} ({len(info['mots'])} mots)")
    print(f"   📝 {info['description']}")
    print(f"   🔤 Échantillon : {', '.join(list(info['mots'])[:8])}...")
    print()

## 4. 🛠️ Gestionnaire Avancé de Stopwords

Classe complète pour gérer, combiner et optimiser vos stopwords personnalisés.

In [ ]:
class GestionnaireStopwords:
    """Gestionnaire avancé pour la création et gestion de stopwords personnalisés."""
    
    def __init__(self, base: str = 'nltk'):
        """
        Initialise le gestionnaire.
        
        Args:
            base: Base de stopwords ('nltk', 'spacy', 'vide')
        """
        self.stopwords = set()
        self.historique = []
        self.domaines_actifs = set()
        
        # Chargement de la base
        if base == 'nltk':
            self.stopwords = set(stopwords.words('french'))
            self.historique.append(f"Base NLTK chargée ({len(self.stopwords)} mots)")
        elif base == 'spacy' and nlp:
            self.stopwords = nlp.Defaults.stop_words.copy()
            self.historique.append(f"Base spaCy chargée ({len(self.stopwords)} mots)")
        else:
            self.historique.append("Gestionnaire initialisé vide")
    
    def ajouter_domaine(self, domaine: str, force: bool = False) -> bool:
        """
        Ajoute les stopwords d'un domaine.
        
        Args:
            domaine: Nom du domaine
            force: Forcer l'ajout même si déjà présent
        
        Returns:
            True si ajouté avec succès
        """
        if domaine not in STOPWORDS_DOMAINES:
            print(f"❌ Domaine '{domaine}' non reconnu")
            print(f"   Domaines disponibles : {list(STOPWORDS_DOMAINES.keys())}")
            return False
        
        if domaine in self.domaines_actifs and not force:
            print(f"⚠️ Domaine '{domaine}' déjà ajouté (utilisez force=True pour forcer)")
            return False
        
        mots_domaine = STOPWORDS_DOMAINES[domaine]['mots']
        ancienne_taille = len(self.stopwords)
        self.stopwords.update(mots_domaine)
        nouveaux_mots = len(self.stopwords) - ancienne_taille
        
        self.domaines_actifs.add(domaine)
        self.historique.append(f"Domaine '{domaine}' ajouté (+{nouveaux_mots} nouveaux mots)")
        print(f"✅ Domaine '{domaine}' ajouté : {nouveaux_mots} nouveaux mots")
        return True
    
    def retirer_domaine(self, domaine: str) -> bool:
        """
        Retire les stopwords d'un domaine.
        
        Args:
            domaine: Nom du domaine à retirer
        
        Returns:
            True si retiré avec succès
        """
        if domaine not in self.domaines_actifs:
            print(f"⚠️ Domaine '{domaine}' pas actuellement actif")
            return False
        
        mots_domaine = STOPWORDS_DOMAINES[domaine]['mots']
        ancienne_taille = len(self.stopwords)
        self.stopwords -= mots_domaine
        mots_retires = ancienne_taille - len(self.stopwords)
        
        self.domaines_actifs.remove(domaine)
        self.historique.append(f"Domaine '{domaine}' retiré (-{mots_retires} mots)")
        print(f"✅ Domaine '{domaine}' retiré : {mots_retires} mots supprimés")
        return True
    
    def ajouter_mots(self, mots: List[str], categorie: str = 'personnalise') -> None:
        """
        Ajoute une liste de mots personnalisés.
        
        Args:
            mots: Liste des mots à ajouter
            categorie: Catégorie pour l'historique
        """
        mots_normalises = [mot.lower().strip() for mot in mots if mot.strip()]
        nouveaux_mots = [mot for mot in mots_normalises if mot not in self.stopwords]
        
        self.stopwords.update(mots_normalises)
        self.historique.append(f"Ajout {categorie} : {len(nouveaux_mots)} nouveaux mots")
        print(f"✅ {len(nouveaux_mots)} nouveaux mots ajoutés ({categorie})")
        
        if len(nouveaux_mots) < len(mots_normalises):
            doublons = len(mots_normalises) - len(nouveaux_mots)
            print(f"   ℹ️ {doublons} mots étaient déjà présents")
    
    def retirer_mots(self, mots: List[str]) -> None:
        """
        Retire une liste de mots.
        
        Args:
            mots: Liste des mots à retirer
        """
        mots_normalises = {mot.lower().strip() for mot in mots if mot.strip()}
        mots_present = mots_normalises.intersection(self.stopwords)
        
        self.stopwords -= mots_normalises
        self.historique.append(f"Suppression : {len(mots_present)} mots retirés")
        print(f"✅ {len(mots_present)} mots supprimés")
        
        if len(mots_present) < len(mots_normalises):
            absents = len(mots_normalises) - len(mots_present)
            print(f"   ℹ️ {absents} mots n'étaient pas présents")
    
    def filtrer_texte(self, texte: str, retourner_stopwords: bool = False) -> List[str] or Tuple[List[str], List[str]]:
        """
        Filtre un texte en supprimant les stopwords.
        
        Args:
            texte: Texte à filtrer
            retourner_stopwords: Si True, retourne aussi les stopwords trouvés
        
        Returns:
            Liste des mots filtrés, optionnellement avec les stopwords trouvés
        """
        mots = re.findall(r'\w+', texte.lower())
        mots_filtres = [mot for mot in mots if mot not in self.stopwords]
        
        if retourner_stopwords:
            stopwords_trouves = [mot for mot in mots if mot in self.stopwords]
            return mots_filtres, stopwords_trouves
        
        return mots_filtres
    
    def analyser_texte(self, texte: str) -> Dict:
        """
        Analyse complète d'un texte avec les stopwords actuels.
        
        Args:
            texte: Texte à analyser
        
        Returns:
            Dictionnaire avec les statistiques
        """
        mots_originaux = re.findall(r'\w+', texte.lower())
        mots_filtres, stopwords_trouves = self.filtrer_texte(texte, retourner_stopwords=True)
        
        return {
            'mots_originaux': len(mots_originaux),
            'mots_filtres': len(mots_filtres),
            'stopwords_supprimes': len(stopwords_trouves),
            'reduction_pourcentage': round((1 - len(mots_filtres)/len(mots_originaux))*100, 1) if mots_originaux else 0,
            'stopwords_uniques': len(set(stopwords_trouves)),
            'vocabulaire_unique_filtre': len(set(mots_filtres)),
            'echantillon_filtres': mots_filtres[:10],
            'echantillon_stopwords': list(set(stopwords_trouves))[:10]
        }
    
    def statistiques(self) -> Dict:
        """
        Retourne les statistiques du gestionnaire.
        
        Returns:
            Dictionnaire avec les statistiques
        """
        return {
            'total_stopwords': len(self.stopwords),
            'domaines_actifs': list(self.domaines_actifs),
            'nb_domaines': len(self.domaines_actifs),
            'historique_operations': len(self.historique),
            'longueur_moyenne': round(sum(len(mot) for mot in self.stopwords) / len(self.stopwords), 2) if self.stopwords else 0,
            'mot_plus_long': max(self.stopwords, key=len) if self.stopwords else None,
            'mot_plus_court': min(self.stopwords, key=len) if self.stopwords else None
        }
    
    def afficher_historique(self) -> None:
        """
        Affiche l'historique des opérations.
        """
        print("📜 HISTORIQUE DES OPÉRATIONS\n")
        for i, operation in enumerate(self.historique, 1):
            print(f"  {i:2d}. {operation}")
    
    def sauvegarder(self, fichier: str) -> None:
        """
        Sauvegarde la configuration dans un fichier JSON.
        
        Args:
            fichier: Chemin du fichier de sauvegarde
        """
        config = {
            'stopwords': sorted(list(self.stopwords)),
            'domaines_actifs': list(self.domaines_actifs),
            'historique': self.historique,
            'statistiques': self.statistiques()
        }
        
        with open(fichier, 'w', encoding='utf-8') as f:
            json.dump(config, f, ensure_ascii=False, indent=2)
        
        print(f"✅ Configuration sauvegardée dans '{fichier}'")
    
    def charger(self, fichier: str) -> None:
        """
        Charge une configuration depuis un fichier JSON.
        
        Args:
            fichier: Chemin du fichier à charger
        """
        try:
            with open(fichier, 'r', encoding='utf-8') as f:
                config = json.load(f)
            
            self.stopwords = set(config['stopwords'])
            self.domaines_actifs = set(config['domaines_actifs'])
            self.historique = config['historique']
            self.historique.append(f"Configuration chargée depuis '{fichier}'")
            
            print(f"✅ Configuration chargée depuis '{fichier}'")
            print(f"   {len(self.stopwords)} stopwords, {len(self.domaines_actifs)} domaines actifs")
            
        except FileNotFoundError:
            print(f"❌ Fichier '{fichier}' non trouvé")
        except json.JSONDecodeError:
            print(f"❌ Erreur de format JSON dans '{fichier}'")

print("✅ Classe GestionnaireStopwords créée")

In [ ]:
# Démonstration du gestionnaire avancé
print("🛠️ DÉMONSTRATION DU GESTIONNAIRE AVANCÉ\n")

# Création d'un gestionnaire pour e-commerce
gestionnaire = GestionnaireStopwords(base='nltk')
print(f"Base NLTK : {gestionnaire.statistiques()['total_stopwords']} stopwords")

# Ajout du domaine e-commerce
gestionnaire.ajouter_domaine('e_commerce')

# Ajout de mots personnalisés
mots_perso = ['franchement', 'sincèrement', 'honnêtement', 'globalement', 'finalement']
gestionnaire.ajouter_mots(mots_perso, 'adverbes_opinion')

# Statistiques finales
stats = gestionnaire.statistiques()
print(f"\n📊 Configuration finale :")
print(f"  Total stopwords : {stats['total_stopwords']}")
print(f"  Domaines actifs : {', '.join(stats['domaines_actifs'])}")
print(f"  Longueur moyenne : {stats['longueur_moyenne']} caractères")
print(f"  Mot le plus long : '{stats['mot_plus_long']}'")
print(f"  Mot le plus court : '{stats['mot_plus_court']}'")

## 5. ✅ Validation et Tests

Comment valider que vos stopwords personnalisés sont efficaces ?

In [ ]:
# Test comparatif sur différents textes
textes_test = {
    "Avis positif e-commerce": """
    Ce produit est vraiment excellent ! Service client très professionnel, 
    livraison ultra rapide. Je recommande vivement cet article à tous. 
    Parfait rapport qualité-prix, franchement je suis très satisfait.
    """,
    
    "Avis négatif e-commerce": """
    Produit décevant, qualité médiocre. Service client pas du tout à l'écoute.
    Livraison tardive et emballage abîmé. Honnêtement, je ne recommande pas 
    du tout ce site. Très mauvaise expérience d'achat.
    """,
    
    "Description technique": """
    Cette fonction permet de configurer les paramètres du système. 
    Elle utilise une méthode avancée pour optimiser les performances.
    Voir la documentation pour plus d'informations sur l'utilisation.
    """
}

# Comparaison avec différentes configurations
configurations = {
    'Standard NLTK': GestionnaireStopwords(base='nltk'),
    'E-commerce': GestionnaireStopwords(base='nltk'),
    'E-commerce + Perso': GestionnaireStopwords(base='nltk')
}

# Configuration des gestionnaires
configurations['E-commerce'].ajouter_domaine('e_commerce')
configurations['E-commerce + Perso'].ajouter_domaine('e_commerce')
configurations['E-commerce + Perso'].ajouter_mots(
    ['franchement', 'sincèrement', 'honnêtement', 'globalement', 'vraiment', 'très']
)

print("🧪 TESTS COMPARATIFS DE VALIDATION\n")

resultats_test = []

for titre_texte, texte in textes_test.items():
    print(f"📄 {titre_texte.upper()}")
    print(f"Texte : {texte.strip()[:80]}...\n")
    
    for nom_config, gestionnaire in configurations.items():
        analyse = gestionnaire.analyser_texte(texte)
        
        resultats_test.append({
            'Texte': titre_texte,
            'Configuration': nom_config,
            'Mots_originaux': analyse['mots_originaux'],
            'Mots_filtres': analyse['mots_filtres'],
            'Reduction_%': analyse['reduction_pourcentage'],
            'Stopwords_uniques': analyse['stopwords_uniques']
        })
        
        print(f"  {nom_config:20s} : {analyse['mots_filtres']:2d} mots ({analyse['reduction_pourcentage']:5.1f}% réduction)")
        print(f"  {'':22s} → {' '.join(analyse['echantillon_filtres'][:6])}...")
    
    print("\n" + "="*70 + "\n")

# Analyse des résultats
df_resultats = pd.DataFrame(resultats_test)
print("📊 TABLEAU RÉCAPITULATIF :")
print(df_resultats.pivot_table(
    index='Texte', 
    columns='Configuration', 
    values='Reduction_%', 
    aggfunc='first'
).round(1))

In [ ]:
# Visualisation des résultats de validation
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Graphique 1: Réduction par configuration
reduction_par_config = df_resultats.groupby('Configuration')['Reduction_%'].mean()
bars1 = axes[0,0].bar(reduction_par_config.index, reduction_par_config.values, 
                      color=['skyblue', 'lightgreen', 'orange'], alpha=0.7)
axes[0,0].set_title('📉 Réduction Moyenne par Configuration', fontweight='bold')
axes[0,0].set_ylabel('Réduction (%)')
axes[0,0].tick_params(axis='x', rotation=45)

# Ajout des valeurs sur les barres
for bar, value in zip(bars1, reduction_par_config.values):
    axes[0,0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
                   f'{value:.1f}%', ha='center', va='bottom', fontweight='bold')

# Graphique 2: Mots conservés par type de texte
textes_order = ['Avis positif e-commerce', 'Avis négatif e-commerce', 'Description technique']
configs = df_resultats['Configuration'].unique()
x = np.arange(len(textes_order))
width = 0.25

for i, config in enumerate(configs):
    data = df_resultats[df_resultats['Configuration'] == config]
    values = [data[data['Texte'] == texte]['Mots_filtres'].iloc[0] for texte in textes_order]
    axes[0,1].bar(x + i*width, values, width, label=config, alpha=0.7)

axes[0,1].set_title('📊 Mots Conservés par Type de Texte', fontweight='bold')
axes[0,1].set_xlabel('Type de texte')
axes[0,1].set_ylabel('Nombre de mots conservés')
axes[0,1].set_xticks(x + width)
axes[0,1].set_xticklabels([t.replace(' e-commerce', '') for t in textes_order], rotation=45)
axes[0,1].legend()

# Graphique 3: Distribution des réductions
for config in configs:
    data = df_resultats[df_resultats['Configuration'] == config]['Reduction_%']
    axes[1,0].hist(data, alpha=0.6, label=config, bins=5)

axes[1,0].set_title('📈 Distribution des Taux de Réduction', fontweight='bold')
axes[1,0].set_xlabel('Réduction (%)')
axes[1,0].set_ylabel('Fréquence')
axes[1,0].legend()

# Graphique 4: Efficacité par domaine
efficacite_data = {
    'E-commerce\n(avis)': df_resultats[
        (df_resultats['Configuration'] == 'E-commerce + Perso') & 
        (df_resultats['Texte'].str.contains('Avis'))
    ]['Reduction_%'].mean(),
    'Technique\n(description)': df_resultats[
        (df_resultats['Configuration'] == 'E-commerce + Perso') & 
        (df_resultats['Texte'].str.contains('technique'))
    ]['Reduction_%'].mean(),
    'Standard\n(moyenne)': df_resultats[
        df_resultats['Configuration'] == 'Standard NLTK'
    ]['Reduction_%'].mean()
}

bars4 = axes[1,1].bar(efficacite_data.keys(), efficacite_data.values(), 
                      color=['lightcoral', 'lightblue', 'lightgray'], alpha=0.7)
axes[1,1].set_title('🎯 Efficacité par Domaine d\'Application', fontweight='bold')
axes[1,1].set_ylabel('Réduction (%)')

# Ajout des valeurs
for bar, value in zip(bars4, efficacite_data.values()):
    if not np.isnan(value):
        axes[1,1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
                       f'{value:.1f}%', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n💡 CONCLUSIONS DE LA VALIDATION :")
print(f"  • La personnalisation améliore l'efficacité de {reduction_par_config['E-commerce + Perso'] - reduction_par_config['Standard NLTK']:.1f} points")
print(f"  • Réduction optimale pour le domaine ciblé : {efficacite_data['E-commerce\n(avis)']:.1f}%")
print(f"  • Les stopwords personnalisés sont plus sélectifs et précis")

## 6. 📊 Optimisation par Fréquence

Techniques pour optimiser automatiquement vos listes de stopwords.

In [ ]:
class OptimisateurStopwords:
    """Optimise automatiquement les listes de stopwords selon des critères."""
    
    def __init__(self, corpus: List[str]):
        """
        Initialise avec un corpus de référence.
        
        Args:
            corpus: Liste de textes pour l'analyse
        """
        self.corpus = corpus
        self.stats = self._analyser_corpus()
    
    def _analyser_corpus(self) -> Dict:
        """
        Analyse statistique complète du corpus.
        
        Returns:
            Dictionnaire avec les statistiques
        """
        tous_mots = []
        frequences_doc = defaultdict(int)
        
        for texte in self.corpus:
            mots = re.findall(r'\w+', texte.lower())
            tous_mots.extend(mots)
            
            # Fréquence par document
            for mot in set(mots):
                frequences_doc[mot] += 1
        
        frequences_globales = Counter(tous_mots)
        
        return {
            'frequences_globales': frequences_globales,
            'frequences_document': frequences_doc,
            'nb_documents': len(self.corpus),
            'nb_mots_total': len(tous_mots),
            'vocabulaire_unique': len(frequences_globales)
        }
    
    def candidats_frequence_elevee(self, percentile: float = 90) -> List[Tuple[str, int, float]]:
        """
        Identifie les mots avec une fréquence très élevée.
        
        Args:
            percentile: Percentile de fréquence (ex: 90 = top 10%)
        
        Returns:
            Liste des candidats avec fréquence et percentile
        """
        frequences = list(self.stats['frequences_globales'].values())
        seuil = np.percentile(frequences, percentile)
        
        candidats = []
        for mot, freq in self.stats['frequences_globales'].items():
            if freq >= seuil:
                pct = (sum(1 for f in frequences if f < freq) / len(frequences)) * 100
                candidats.append((mot, freq, pct))
        
        candidats.sort(key=lambda x: x[1], reverse=True)
        return candidats
    
    def candidats_ubiquite(self, seuil_presence: float = 0.8) -> List[Tuple[str, float, int]]:
        """
        Identifie les mots présents dans beaucoup de documents.
        
        Args:
            seuil_presence: Pourcentage minimum de documents
        
        Returns:
            Liste des candidats avec taux de présence
        """
        candidats = []
        nb_docs = self.stats['nb_documents']
        
        for mot, doc_freq in self.stats['frequences_document'].items():
            taux_presence = doc_freq / nb_docs
            if taux_presence >= seuil_presence:
                freq_globale = self.stats['frequences_globales'][mot]
                candidats.append((mot, taux_presence, freq_globale))
        
        candidats.sort(key=lambda x: x[1], reverse=True)
        return candidats
    
    def score_stopword(self, mot: str) -> float:
        """
        Calcule un score de "stopword-ness" pour un mot.
        
        Args:
            mot: Mot à évaluer
        
        Returns:
            Score entre 0 et 1 (1 = très probable stopword)
        """
        if mot not in self.stats['frequences_globales']:
            return 0.0
        
        freq_globale = self.stats['frequences_globales'][mot]
        freq_doc = self.stats['frequences_document'][mot]
        
        # Normalisation
        freq_norm = freq_globale / self.stats['nb_mots_total']
        ubiquite_norm = freq_doc / self.stats['nb_documents']
        longueur_norm = max(0, (5 - len(mot)) / 5)  # Mots courts = score plus élevé
        
        # Score composé (pondération personnalisable)
        score = (freq_norm * 0.4) + (ubiquite_norm * 0.4) + (longueur_norm * 0.2)
        
        return min(1.0, score)
    
    def optimiser_liste(self, stopwords_actuels: Set[str], 
                       seuil_ajout: float = 0.7, 
                       seuil_suppression: float = 0.3) -> Tuple[Set[str], Set[str]]:
        """
        Optimise une liste de stopwords existante.
        
        Args:
            stopwords_actuels: Liste actuelle des stopwords
            seuil_ajout: Score minimum pour ajouter un mot
            seuil_suppression: Score maximum pour garder un mot
        
        Returns:
            Tuple (mots_a_ajouter, mots_a_supprimer)
        """
        mots_a_ajouter = set()
        mots_a_supprimer = set()
        
        # Candidats à ajouter
        for mot in self.stats['frequences_globales']:
            if mot not in stopwords_actuels:
                score = self.score_stopword(mot)
                if score >= seuil_ajout:
                    mots_a_ajouter.add(mot)
        
        # Candidats à supprimer
        for mot in stopwords_actuels:
            if mot in self.stats['frequences_globales']:
                score = self.score_stopword(mot)
                if score <= seuil_suppression:
                    mots_a_supprimer.add(mot)
        
        return mots_a_ajouter, mots_a_supprimer
    
    def rapport_optimisation(self, stopwords_actuels: Set[str]) -> None:
        """
        Génère un rapport d'optimisation détaillé.
        
        Args:
            stopwords_actuels: Liste actuelle des stopwords
        """
        mots_a_ajouter, mots_a_supprimer = self.optimiser_liste(stopwords_actuels)
        
        print("🔧 RAPPORT D'OPTIMISATION\n")
        print(f"📊 Statistiques du corpus :")
        print(f"  • Documents : {self.stats['nb_documents']}")
        print(f"  • Mots total : {self.stats['nb_mots_total']:,}")
        print(f"  • Vocabulaire unique : {self.stats['vocabulaire_unique']:,}")
        
        print(f"\n🎯 Recommandations :")
        print(f"  • Stopwords actuels : {len(stopwords_actuels)}")
        print(f"  • À ajouter : {len(mots_a_ajouter)}")
        print(f"  • À supprimer : {len(mots_a_supprimer)}")
        print(f"  • Total optimisé : {len(stopwords_actuels) + len(mots_a_ajouter) - len(mots_a_supprimer)}")
        
        if mots_a_ajouter:
            print(f"\n➕ Mots à ajouter (score élevé) :")
            for mot in sorted(mots_a_ajouter, key=lambda x: self.score_stopword(x), reverse=True)[:10]:
                score = self.score_stopword(mot)
                freq = self.stats['frequences_globales'][mot]
                print(f"  '{mot}' (score: {score:.3f}, fréq: {freq})")
        
        if mots_a_supprimer:
            print(f"\n➖ Mots à supprimer (score faible) :")
            for mot in sorted(mots_a_supprimer, key=lambda x: self.score_stopword(x))[:10]:
                score = self.score_stopword(mot)
                freq = self.stats['frequences_globales'].get(mot, 0)
                print(f"  '{mot}' (score: {score:.3f}, fréq: {freq})")

print("✅ Classe OptimisateurStopwords créée")

In [ ]:
# Test de l'optimisateur sur notre corpus e-commerce
corpus_ecommerce_etendu = [
    "Ce produit est vraiment excellent, je le recommande à tous mes amis.",
    "Service client très professionnel, livraison rapide, emballage soigné.",
    "Produit conforme à la description, qualité au rendez-vous, très satisfait.",
    "Prix un peu élevé mais la qualité justifie largement cet investissement.",
    "Commande reçue rapidement, produit de bonne qualité, service réactif.",
    "Très bon rapport qualité prix, produit fiable, livraison dans les temps.",
    "Service après vente excellent, problème résolu rapidement, équipe professionnelle.",
    "Produit exactement comme décrit, emballage parfait, livraison ultra rapide.",
    "Qualité exceptionnelle, service client à l'écoute, je recommande vivement.",
    "Achat sans problème, produit conforme, délai de livraison respecté.",
    "Déçu par ce produit, qualité décevante, service client pas à l'écoute.",
    "Livraison tardive, emballage abîmé, produit endommagé, très déçu.",
    "Service client incompétent, problème non résolu, je déconseille.",
    "Produit ne correspond pas à la description, qualité médiocre.",
    "Commande annulée sans préavis, service client injoignable, scandaleux."
]

# Création de l'optimisateur
optimisateur = OptimisateurStopwords(corpus_ecommerce_etendu)

# Test avec les stopwords NLTK de base
stopwords_base = set(stopwords.words('french'))
optimisateur.rapport_optimisation(stopwords_base)

print("\n" + "="*70)

# Application des optimisations
mots_a_ajouter, mots_a_supprimer = optimisateur.optimiser_liste(stopwords_base)
stopwords_optimises = (stopwords_base | mots_a_ajouter) - mots_a_supprimer

print(f"\n✨ RÉSULTATS DE L'OPTIMISATION :")
print(f"  • Stopwords optimisés : {len(stopwords_optimises)} (+{len(mots_a_ajouter)-len(mots_a_supprimer)})")
print(f"  • Efficacité théorique améliorée pour ce corpus")

## 7. 🎪 Cas d'Usage Pratiques

Exemples concrets d'application dans différents contextes.

In [ ]:
# Cas d'usage 1: Analyse de sentiment e-commerce
print("🎪 CAS D'USAGE 1 : ANALYSE DE SENTIMENT E-COMMERCE\n")

# Données d'exemple
avis_clients = [
    ("Produit fantastique, livraison rapide, service client excellent!", "positif"),
    ("Très déçu, qualité médiocre, je ne recommande absolument pas.", "négatif"),
    ("Correct sans plus, ça fait le travail mais rien d'exceptionnel.", "neutre"),
    ("Parfait! Exactement ce que je cherchais, très satisfait de mon achat.", "positif"),
    ("Service après-vente inexistant, produit défaillant, très mauvaise expérience.", "négatif")
]

# Configuration spécialisée pour l'analyse de sentiment
gestionnaire_sentiment = GestionnaireStopwords(base='nltk')
gestionnaire_sentiment.ajouter_domaine('e_commerce')

# Ajout de mots spécifiques au sentiment (mais non informatifs)
mots_sentiment_generiques = [
    'produit', 'service', 'client', 'livraison', 'achat', 'commande',
    'très', 'vraiment', 'super', 'assez', 'plutôt', 'franchement'
]
gestionnaire_sentiment.ajouter_mots(mots_sentiment_generiques, 'sentiment_generiques')

print("Configuration pour analyse de sentiment :")
print(f"  • {gestionnaire_sentiment.statistiques()['total_stopwords']} stopwords")
print(f"  • Domaines : {', '.join(gestionnaire_sentiment.statistiques()['domaines_actifs'])}")

print("\n📊 Résultats du filtrage :")
for avis, sentiment in avis_clients:
    mots_filtres = gestionnaire_sentiment.filtrer_texte(avis)
    print(f"  {sentiment:8s} : {' '.join(mots_filtres[:6])}...")

In [ ]:
# Cas d'usage 2: Classification automatique de tickets support
print("🎪 CAS D'USAGE 2 : CLASSIFICATION DE TICKETS SUPPORT\n")

tickets_support = [
    ("Mon compte est bloqué, je n'arrive plus à me connecter depuis hier.", "compte"),
    ("Le paiement n'a pas fonctionné, erreur lors de la validation de ma carte.", "paiement"),
    ("Ma commande n'est toujours pas arrivée, pouvez-vous vérifier le statut?", "livraison"),
    ("Le produit reçu est défectueux, je souhaite un remboursement.", "produit"),
    ("Comment modifier mes informations personnelles dans mon profil?", "compte")
]

# Configuration pour classification de tickets
gestionnaire_tickets = GestionnaireStopwords(base='nltk')

# Stopwords spécifiques au support client
mots_support = [
    'pouvez', 'vous', 'comment', 'pourquoi', 'depuis', 'toujours',
    'problème', 'souhaite', 'vérifier', 'modifier', 'information'
]
gestionnaire_tickets.ajouter_mots(mots_support, 'support_client')

print("Configuration pour tickets support :")
print(f"  • {gestionnaire_tickets.statistiques()['total_stopwords']} stopwords")

print("\n🎯 Mots-clés extraits pour classification :")
for ticket, categorie in tickets_support:
    mots_cles = gestionnaire_tickets.filtrer_texte(ticket)
    print(f"  {categorie:10s} : {' '.join(mots_cles[:5])}...")
    print(f"  {'':12s} → Ticket: {ticket[:50]}...")
    print()

In [ ]:
# Cas d'usage 3: Analyse de réseaux sociaux
print("🎪 CAS D'USAGE 3 : ANALYSE DE RÉSEAUX SOCIAUX\n")

posts_sociaux = [
    "Salut les amis! Alors, qu'est-ce que vous pensez du nouveau film? Moi je l'ai trouvé génial!",
    "MDR cette vidéo! Genre trop drôle, je suis mort de rire 😂",
    "Bon alors, qui vient à la soirée ce soir? On va bien s'amuser!",
    "Franchement, ce truc est ouf! Vous devez absolument essayer.",
    "Bof... pas terrible cette série. J'ai regardé 2 épisodes, ça m'a saoulé."
]

# Configuration pour réseaux sociaux
gestionnaire_social = GestionnaireStopwords(base='nltk')
gestionnaire_social.ajouter_domaine('reseaux_sociaux')

# Ajout d'expressions typiques des réseaux sociaux
expressions_sociales = [
    'vous', 'pensez', 'trouvé', 'regardé', 'vient', 'devez', 'alors',
    'bien', 'pas', 'cette', 'film', 'vidéo', 'soirée', 'série'
]
gestionnaire_social.ajouter_mots(expressions_sociales, 'expressions_sociales')

print("Configuration pour réseaux sociaux :")
print(f"  • {gestionnaire_social.statistiques()['total_stopwords']} stopwords")
print(f"  • Spécialisé pour le langage informel")

print("\n💬 Contenu filtré (focus sur l'opinion/émotion) :")
for i, post in enumerate(posts_sociaux, 1):
    mots_pertinents = gestionnaire_social.filtrer_texte(post)
    print(f"  Post {i}: {' '.join(mots_pertinents[:8])}")
    print(f"  Original: {post[:60]}...")
    print()

## 8. 💾 Export et Réutilisation

Comment sauvegarder, partager et réutiliser vos configurations de stopwords.

In [ ]:
# Création de configurations prêtes à l'emploi
configurations_preetes = {
    'e_commerce_avis': {
        'description': 'Optimisé pour l\'analyse d\'avis clients e-commerce',
        'base': 'nltk',
        'domaines': ['e_commerce'],
        'mots_supplementaires': ['vraiment', 'très', 'super', 'excellent', 'parfait', 'génial']
    },
    
    'support_client': {
        'description': 'Spécialisé pour la classification de tickets support',
        'base': 'nltk',
        'domaines': ['technique'],
        'mots_supplementaires': ['problème', 'aide', 'question', 'comment', 'pourquoi']
    },
    
    'reseaux_sociaux_fr': {
        'description': 'Adapté aux réseaux sociaux français (argot, abréviations)',
        'base': 'nltk',
        'domaines': ['reseaux_sociaux'],
        'mots_supplementaires': ['alors', 'bon', 'bref', 'enfin']
    },
    
    'medical_minimal': {
        'description': 'Version légère pour textes médicaux (garde termes techniques)',
        'base': 'nltk',
        'domaines': [],
        'mots_supplementaires': ['très', 'assez', 'plutôt', 'bien', 'mal']
    }
}

def creer_configuration(nom_config: str) -> GestionnaireStopwords:
    """
    Crée un gestionnaire selon une configuration prédéfinie.
    
    Args:
        nom_config: Nom de la configuration
    
    Returns:
        Gestionnaire configuré
    """
    if nom_config not in configurations_preetes:
        raise ValueError(f"Configuration '{nom_config}' inconnue")
    
    config = configurations_preetes[nom_config]
    gestionnaire = GestionnaireStopwords(base=config['base'])
    
    # Ajout des domaines
    for domaine in config['domaines']:
        gestionnaire.ajouter_domaine(domaine)
    
    # Ajout des mots supplémentaires
    if config['mots_supplementaires']:
        gestionnaire.ajouter_mots(config['mots_supplementaires'], 'configuration_preete')
    
    return gestionnaire

# Test des configurations prêtes à l'emploi
print("💾 CONFIGURATIONS PRÊTES À L'EMPLOI\n")

for nom, config in configurations_preetes.items():
    gestionnaire = creer_configuration(nom)
    stats = gestionnaire.statistiques()
    
    print(f"🔧 {nom.upper().replace('_', ' ')}")
    print(f"   📝 {config['description']}")
    print(f"   📊 {stats['total_stopwords']} stopwords")
    if stats['domaines_actifs']:
        print(f"   🏢 Domaines : {', '.join(stats['domaines_actifs'])}")
    print()

In [ ]:
# Sauvegarde et chargement des configurations
print("💾 SAUVEGARDE ET CHARGEMENT\n")

# Création d'une configuration personnalisée
mon_gestionnaire = GestionnaireStopwords(base='nltk')
mon_gestionnaire.ajouter_domaine('e_commerce')
mon_gestionnaire.ajouter_mots(['absolument', 'certainement', 'effectivement'], 'adverbes_certitude')

# Sauvegarde
fichier_config = 'ma_config_stopwords.json'
mon_gestionnaire.sauvegarder(fichier_config)

# Test de chargement
nouveau_gestionnaire = GestionnaireStopwords(base='vide')
nouveau_gestionnaire.charger(fichier_config)

print(f"\n📋 Vérification du chargement :")
print(f"  • Stopwords chargés : {nouveau_gestionnaire.statistiques()['total_stopwords']}")
print(f"  • Domaines actifs : {', '.join(nouveau_gestionnaire.statistiques()['domaines_actifs'])}")

# Affichage de l'historique
print("\n📜 Historique des opérations :")
for i, operation in enumerate(nouveau_gestionnaire.historique[-5:], 1):
    print(f"  {i}. {operation}")

In [ ]:
# Fonction utilitaire pour export vers différents formats
def exporter_stopwords(gestionnaire: GestionnaireStopwords, format_export: str = 'txt') -> str:
    """
    Exporte les stopwords vers différents formats.
    
    Args:
        gestionnaire: Gestionnaire de stopwords
        format_export: Format ('txt', 'csv', 'python', 'json')
    
    Returns:
        Contenu formaté pour l'export
    """
    stopwords_list = sorted(list(gestionnaire.stopwords))
    stats = gestionnaire.statistiques()
    
    if format_export == 'txt':
        header = f"# Stopwords personnalisés - {stats['total_stopwords']} mots\n"
        header += f"# Domaines: {', '.join(stats['domaines_actifs'])}\n\n"
        return header + '\n'.join(stopwords_list)
    
    elif format_export == 'csv':
        return 'stopword\n' + '\n'.join(stopwords_list)
    
    elif format_export == 'python':
        return f"""# Configuration stopwords générée automatiquement
# Total: {stats['total_stopwords']} mots
# Domaines: {', '.join(stats['domaines_actifs'])}

STOPWORDS_PERSONNALISES = {{
    {', '.join([f"'{mot}'" for mot in stopwords_list[:10]])},
    # ... {stats['total_stopwords'] - 10} autres mots
}}
"""
    
    elif format_export == 'json':
        return json.dumps({
            'stopwords': stopwords_list,
            'metadata': {
                'total': stats['total_stopwords'],
                'domaines': stats['domaines_actifs'],
                'export_date': str(pd.Timestamp.now())
            }
        }, ensure_ascii=False, indent=2)

# Test des exports
print("📤 EXEMPLES D'EXPORT\n")

# Export Python
export_python = exporter_stopwords(mon_gestionnaire, 'python')
print("🐍 Export Python :")
print(export_python[:300] + "...")

print("\n" + "="*50)

# Export JSON
export_json = exporter_stopwords(mon_gestionnaire, 'json')
print("\n📋 Export JSON (extrait) :")
print(export_json[:300] + "...")

print("\n✅ Tous les formats d'export sont disponibles !")

## 🎉 Conclusion et Bonnes Pratiques

### ✅ Ce que vous avez appris :

1. **Identifier automatiquement** les candidats stopwords dans vos données
2. **Créer des listes spécialisées** par domaine d'application
3. **Gérer efficacement** vos configurations avec le GestionnaireStopwords
4. **Optimiser automatiquement** vos listes selon des critères statistiques
5. **Valider l'efficacité** de vos stopwords personnalisés
6. **Appliquer concrètement** dans différents cas d'usage
7. **Sauvegarder et réutiliser** vos configurations

### 💡 Bonnes Pratiques Finales :

#### ✅ À FAIRE :
- **Analyser votre corpus** avant de créer des stopwords
- **Tester différentes configurations** et mesurer l'impact
- **Documenter vos choix** et garder un historique
- **Adapter selon le contexte** (domaine, objectif, type de texte)
- **Valider sur des échantillons** manuellement
- **Itérer et améliorer** selon les résultats obtenus

#### ❌ À ÉVITER :
- Utiliser la même liste pour tous les projets
- Supprimer des mots importants pour votre analyse
- Ignorer les spécificités de votre domaine
- Ne pas mesurer l'impact sur les performances finales
- Oublier de sauvegarder vos configurations optimisées

### 🚀 Prochaines Étapes :

Maintenant que vous maîtrisez les stopwords personnalisés, vous êtes prêt pour :
- **Lemmatisation avancée** avec des dictionnaires spécialisés
- **Pipeline de preprocessing complet** intégrant toutes les techniques
- **Évaluation quantitative** de la qualité du preprocessing

---

**🎯 Message final :** Les stopwords ne sont pas une science exacte ! C'est un équilibre entre suppression du bruit et conservation de l'information. Expérimentez, mesurez, adaptez !